In [1]:
#All Code Written by Gary Zeri
#Chapman University Computer Science Major, Member of the LaRue CatLab

#Import All Required Packages Here
import ipywidgets as widgets
from IPython.display import clear_output
from diatomicPotentials import extendedRydberg
from RKRClass import RKR

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#May be changing later 
import numpy as np
from BasisSets import HOW
from scipy.integrate import quad as integrate
from scipy.misc import derivative as ddx
from scipy.linalg import eigh
from tqdm import tqdm

In [2]:
#Declare All Global Variables Here
figure = {
        "data": [],
        "layout":
        {
           "xaxis":{"title":"Bond Distance"},
           "yaxis":{"title":"Energy"},
           "title":{"text":"Potential Energy Curves"}
        },    
    }    

rydberg = extendedRydberg()
rkr = RKR()

In [3]:
def waveFunction(r, v, eVec, basisFunctions):
    answer = 0
    
    for index, basis in enumerate(basisFunctions):
        answer += eVec[v, index] * basis(r)
    
    return float(answer)

##########################################################################################

def potentialAnalysis():
    
    print("Starting Vibrational Energy Calculation")
    
    #Set Up Basis
    basisSize = 15
    u = (12*16) / (12+16)
    re = 1.128323
    we = 1904.20 
    
    basis = HOW(re, we, u, basisSize )
    
    print("Computing T")
    #Compute T, the Kinetic Energy
    T= np.zeros([basis.size, basis.size])
    
    for b1 in range(basis.size):
        for b2 in range(basis.size):

            if(b1 == b2):
                t = 2*b1 + 1
            elif(b1 == b2 + 2):
                t = -np.sqrt( b1 * (b1 - 1) )
            elif(b1 == b2 - 2):
                t = -np.sqrt( (b1+1) * (b1+2)  )
            else:
                t = 0
            T[b1, b2] += round(t, 4)    

    #Multiply by RKR Omega to have units of 1/cm
    #Basis Omega has units of Hertz
    #Should divison be by 4 or by 25
    T *= rkr.we / 4
 
    print("Computing V")
    #Compute the Potential Energy, V
    V = np.zeros([basis.size, basis.size])

    print("Computing V(r)")
    for index1 in tqdm(range(basis.size)):
        b1 = basis.basis[index1]
        for index2, b2 in enumerate(basis.basis):

            integrand = lambda r : b1(r) * rydberg.equation(r) * b2(r)
            #Verified that both Scipy and MPMath have the same accuracy
            V[index1, index2] += integrate(integrand, 0, np.inf, epsrel=pow(10,-100), limit = 200)[0]
            #V[index1, index2] += mpmath.quad(integrand, [0, mpmath.inf])
    
    H = V + T
    
    eVal, eVec = eigh(H)
    
    #Add Vibrational Energy Levels to the Graph
    figure["data"].append(
       {
            "type":"scatter",
            "x":[1.3] * len(eVal),
            "y":[float(e) for e in eVal],
            "connectgaps":False,
            "mode":"markers",
            "name":"Vibrational Energy Levels"
        }
    )
    
    x = np.arange(0, 3, .01)
    for v in tqdm(range(len(basis.basis))):
        y = [ waveFunction(r, v, eVec, basis.basis) for r in x  ]

        figure["data"].append(
            {
                "x":x,
                "y":y,
                "name":str(v)
            }
        )
    
    iplot(figure)

##########################################################################################

def plot(x, y, name):
    
    figure["data"].append(
         {
            "type":"scatter",
            "x":x,
            "y":y,
            "connectgaps":True,
            "mode":"markers", 
            "name":name,
            #"marker":{"color":"blue"}
        }
    )
    
##########################################################################################    

#builds the Extended-Rydberg fit for the given radius and energy data
def buildPotential(R, E):
        
    print("Building Rydberg Potential Fit")
    rydberg.fitPotential(R, E)
    
    print("Graphing Rydberg Potential")
    ER = rydberg.graphData(0, R[-1] + 4)
    
    plot(ER[0], ER[1], "Extended Rydberg Fit")
        
##########################################################################################    

def loadFile():
    
    global fileLoad
        
    fileLoad = widgets.FileUpload(
        accept = ".txt",
        multiple = True,
        button_style = "info"
    )
        
    display(fileLoad)
    
    #reset graph data if new file is uploaded
    figure["data"] = []
        
    fileLoad.observe(parsePotentialFile, "value")

##########################################################################################    
    
def parsePotentialFile(file):
    
    clear_output()
    display(choice)
    loadFile()
    
    #Remove all unneded data so that only used data is accessible from the file dictionary
    file = file["new"][list(file["new"].keys())[0]]
    
    #check that file name is greater than five characters to ensure that [-4:] 
    #command will not cause an error when checking that file ends in ".txt"
    if(len(file["metadata"]["name"]) <= 5 or file["metadata"]["name"][-4:] != ".txt"):
            print("Warning! " + file["metadata"]["name"] + "' is not of the '.txt' type!")
    else:
        fileData = file["content"].decode("utf-8").split("\n")
                
        #set up empty lists to store bond distance and respective energy
        global R, E
        R = []
        E = []
        
        #use try catch in case that file is incorrectly formatted
        #and to catch the resulting error from parsing an incorrect file
        #assumed file is formatted with two columns of data separated by a space
        #1st column is bond distance, while 2nd column is the energy at that bond distance
        try:
            for lineNumber, line in enumerate(fileData):
                if(line.isspace() or line == ""):
                    continue

                line = line.split(" ")               

                R.append(float(line[0]))
                E.append(float(line[-1]))
                
        except ValueError:
            print("Warning!! Linenumber " + str(lineNumber) + " in the input file was incorrectly formatted!")
            print("Line " + str(lineNumber) + ": ''" + " ".join(line) + "''")
            print(line)
            return
        
        print("Potential Energy Surface Data Successfully Parsed!")
        plot(R, E, "Potential Energy Surface From File")
        
        buildPotential(R, E)
        #iplot(figure)
        potentialAnalysis()
          
##########################################################################################

#Handles logic for building potential curve from the diatomic constants
def diatomicConstantsLogic(buttonData):
    
    clear_output()
    displayDiatomicConstantsMenu()
    
    R, E = rkr.graphData(resolution = .1, endPoint=99)
    
    if not any(filter(np.isnan, R)):
        plot(R, E, "RKR Potential")
        buildPotential(R, E)
        iplot(figure)
    else:
        print("RKR Computation Failed!")
        print("Please Ensure Your Diatomic Constant Values Are Correct!")
        print("Or Contact zeri@chapman.edu To Report The Error")
    
##########################################################################################

def displayDiatomicConstantsMenu():
    
    rkrWidgets = rkr.widgetInput()
    
    display(rkrWidgets[0])
    display(rkrWidgets[1])
    display(startButton)
        
    startButton.on_click(diatomicConstantsLogic)
    
##########################################################################################

#Main Function used to exeute main decision logic of the program
def main(buttonData):

    #Refreshes screen to ensure buttons do not pile up on top of each other
    clear_output()
    display(choice)
    
    #check whether to display the load file menu or the get constatnts menu
    if(buttonData["new"] == "Use File"):
        loadFile()
    else:
        global startButton
        startButton = widgets.Button(description = "Start Calculation", button_style="Info")
        
        displayDiatomicConstantsMenu()

In [4]:
#Run Functions with Jupyter Widgets Overlay Here

print("Please choose how you like to set up the Extended-Rydberg Equation:")
choice = widgets.ToggleButtons(
    options = ["Use File", "Use Diatomic Constants"],
    button_style = "info",
    tooltips = ["", "Use Diatomic Constants"],
    value = None,
)

basisSize = widgets.IntText()

display(choice)

choice.observe(main, "value")

interactive(children=(FloatText(value=0.0003187, description='$alpha_e$ in $cm^{-1}$'), FloatText(value=0.0821…

interactive(children=(FloatText(value=1.0, description='$\\mu$ in AMU'), Output()), _dom_classes=('widget-inte…

Button(button_style='info', description='Start Calculation', style=ButtonStyle())


Generating RKR Potential


100%|███████████████████████████████████████████████████████████████████████████████| 995/995 [00:03<00:00, 270.23it/s]


Building Rydberg Potential Fit
TEST TEST
19666.952518125006
14.332479054747987
0.4850054618688738
0.03713129815868238
1.2657250297964826e-07
19662.061587489694
Graphing Rydberg Potential
